In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.models import Sequential 
from keras.utils import np_utils
from keras.layers import Dense,Activation,Dropout,LSTM,BatchNormalization
from keras.layers import Flatten
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import to_categorical
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D

In [ ]:
trainset='../input/food-dataset2/Train'

In [ ]:
data_generator = ImageDataGenerator(rescale=1./255,validation_split=0.1,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [ ]:
train_dataset=data_generator.flow_from_directory(trainset,
                                     target_size=(100,100),
                                     batch_size=128,
                                     class_mode='categorical',
                                     subset="training",
                                     shuffle=True,)
validation_set=data_generator.flow_from_directory(trainset,
                                              target_size=(100,100),
                                              batch_size=128,
                                              class_mode='categorical',
                                              subset="validation",
                                              shuffle=True)

In [ ]:
validation_set.class_indices

In [ ]:
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',padding ='same',input_shape=(100,100,3)))
model.add(Conv2D(32,(3,3),activation='relu',kernel_initializer='he_uniform',padding ='same'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform',padding ='same'))
model.add(Conv2D(64,(3,3),activation='relu',kernel_initializer='he_uniform',padding ='same'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),activation='relu',kernel_initializer='he_uniform',padding ='same'))
model.add(Conv2D(128,(3,3),activation='relu',kernel_initializer='he_uniform',padding ='same'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(256,activation='relu',kernel_initializer='he_uniform'))
model.add(Dropout(0.2))
model.add(Dense(10,activation='softmax'))

In [ ]:

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(train_dataset,batch_size=128,epochs=40,verbose=1,validation_data=validation_set)

In [ ]:
score=model.evaluate(validation_set,verbose=1)
print("test accuracy = ",score[1])

In [ ]:
model.save('./food_final.h5')

In [ ]:
model_food=load_model('../input/food-cnn/food_final.h5')

In [ ]:
classes = ['Bánh Bèo','Bánh Mì','Bún Bò Huế','BĐMT','Bún Riêu','Cháo lòng','Cơm tấm','Gỏi cuốn','Hủ tiếu','Pho']


In [ ]:
url='../input/anhmang/banh mi.jfif'
img=load_img(url,target_size=(100,100))
plt.imshow(img)
plt.xticks([])
plt.yticks([])
img=img_to_array(img)
img=img.reshape(1,100,100,3)
img=img/255.0
print(classes[np.argmax(model.predict(img))])

In [ ]:
url='../input/anhmang/bunbo.jpg'
img=load_img(url,target_size=(100,100))
plt.imshow(img)
plt.xticks([])
plt.yticks([])
img=img_to_array(img)
img=img.reshape(1,100,100,3)
img=img/255.0
print(classes[np.argmax(model.predict(img))])

In [ ]:
url='../input/anhmang/goicuon.jpg'
img=load_img(url,target_size=(100,100))
plt.imshow(img)
plt.xticks([])
plt.yticks([])
img=img_to_array(img)
img=img.reshape(1,100,100,3)
img=img/255.0
print(classes[np.argmax(model.predict(img))])

In [ ]:
test_url= '../input/test-food/test'
test=ImageDataGenerator(rescale=1./255,width_shift_range=0.2,height_shift_range=0.2,zoom_range=0.2,horizontal_flip=True)

In [ ]:
test_data=test.flow_from_directory(
    test_url,
    target_size=(100,100),
    batch_size=128,
    class_mode='categorical',
    shuffle=False, 
   )

In [ ]:
results={ 0:'banh beo',1:'banh mi',2:'bun bo',3:'BĐMT',4:'bun rieu',5:'chao long',6:'com tam',7:'goi cuon',8:'hu tieu',9:'pho'}
pred = model.predict_generator(test_data)
plt.figure(figsize=(16,16))
for i in range(pred.shape[0]):
  plt.subplot(5,6,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(load_img(test_data.filepaths[i]))
  plt.xlabel(results[np.argmax(pred[i])])
plt.show()